# Module9_1: Variant Calling

Grab an interactive node for the class 

`qsub -I -l nodes=1:ppn=8 -l walltime=4:00:00 -q hotel`

**TIP** if you start overwriting your own line, use:

`eval $(resize)`

---
### Prerequisites

* access to UNIX terminal (in case the local desktop/laptop is not large enough)
* knowledge of screen command. Attach, detach a screen
* knowledge of basic bash piping and commands such as 
ls, ls -l, cd, mkdir, sort, uniq, cut, grep, more, less, head, tail
* knowledge of one liners like sed 's/x/y/g' or awk '{$==x}'
---

### Create a results folder in your scratch directory and move to this directory
```
mkdir -p ~/scratch/Module_9/results
cd ~/scratch/Module_9/results
```

---
## BAM and coverage
```
samtools flagstat ~/biom262_2019/Module_9/materials/CPTRES4.chr21.bam > CPTRES4.chr21.stat
```

#### Sort and index the bam file
```
samtools sort -m 2G -O bam -T tmp ~/biom262_2019/Module_9/materials/CPTRES4.chr21.bam > CPTRES4.chr21.sorted.bam

samtools index CPTRES4.chr21.sorted.bam
```

#### Coverage via samtools 

```
samtools depth -b ~/biom262_2019/Module_9/materials/Sureselect.chr21.bed -a CPTRES4.chr21.sorted.bam > CPTRES4.chr21.cov

awk '{sum+=$3} END {print sum/NR}' CPTRES4.chr21.cov
```

#### Coverage via picard CollectHSMetrics

```
java -Xmx750m -Xmx1000m  -XX:ParallelGCThreads=1 -jar ~/software/picard.jar CollectHsMetrics \
BAIT_INTERVALS=~/biom262_2019/Module_9/materials/Sureselect.chr21.2.interval_list TARGET_INTERVALS=~/biom262_2019/Module_9/materials/Sureselect.chr21.2.interval_list \
INPUT=CPTRES4.chr21.sorted.bam OUTPUT=CPTRES4.chr21.hsmetrics.txt
```

#### Coverage over a specific gene 


Option 1:
```
bedtools coverage -a ~/biom262_2019/Module_9/resources/CGC.exons.bed -b CPTRES4.chr21.sorted.bam -hist | grep 'RUNX1' | awk '$5>20' | awk '{sum+=$6} END {print sum}'
```

Option 2:
```
grep 'RUNX1' ~/biom262_2019/Module_9/resources/CGC.exons.bed | samtools depth -b - -a CPTRES4.chr21.sorted.bam | awk '$3>20' | wc -l 
```